06.Design Pattern with First-Class Functions

In [30]:
#예제1

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [31]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [33]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [34]:
class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [36]:
class Promotion(ABC):
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod
    def discount(self, order):
        """할인액을 구하고 할인된 가격을 반환"""
        pass

In [45]:
class FidelityPromo(Promotion):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [46]:
#예제2

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [47]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [48]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [49]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [51]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [52]:
#예제3

class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [53]:
def fidelity_promo(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [54]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [55]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [56]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [57]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [58]:
#예제4

promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [59]:
#예제5

Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [60]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [61]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [62]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', '_i11', '_i12', '_i13', '_i14', 'deco', 'target', '_i15', '_15', '_i16', 'register', 'f1', 'f2', 'f3', 'main', '_i17', 'registry', '_i18', 'promos', 'promotion', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', 'best_promo', '_i19', '_i20', 'b', '_i21', '_i22', '_i23', '_23', '_i24', '_i25', '_i26', 'Averager', '_i27', 'avg', '_27', '_i28', '_28', '_i29', '_29', '_i30', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i31', 'park', '_31', '_i32', 'LineItem', '_i33', '_i34', 'Order', '_i35', '_i36', 'Promotion', '_i37', 'FidelityPromo', '_i38', 'joe', 'ann', 'cart', '_38', '_i39', '_39', '_i40', 'banana_cart', '_i41', '_i42', '_42', '_i43', '_43', '_i44', '_i45', 'BulkItemPromo', 'LargeOrderPromo',

In [63]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [64]:
#예제6

promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

07.Function Decorators and Closures

In [14]:
#예제1

def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [15]:
target

<function __main__.deco.<locals>.inner()>

In [17]:
#예제2

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7fac0bcce170>)
running register(<function f2 at 0x7fac0bcce320>)
running main()
registry -> [<function f1 at 0x7fac0bcce170>, <function f2 at 0x7fac0bcce320>]
running f1()
running f2()
running f3()


In [18]:
#예제3

promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [19]:
#예제4

def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [20]:
b=6
f1(3)

3
6


In [21]:
#예제5

b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [22]:
#예제6
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [23]:
b

9

In [24]:
f3(3)

3
9


In [25]:
b=30
f3(3)

3
30


In [26]:
#예제7

class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [27]:
avg = Averager()
avg(10)

10.0

In [28]:
avg(11)

10.5

In [29]:
avg(12)

11.0

In [65]:
#예제8

def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [66]:
avg = make_averager()
avg(10)

10.0

In [67]:
avg(11)

10.5

In [68]:
avg(12)

11.0

In [69]:
#예제9

avg.__code__.co_varnames

('new_value', 'total')

In [70]:
avg.__code__.co_freevars

('series',)

In [71]:
avg.__closure__

(<cell at 0x7fabff168640: list object at 0x7fabfedc2f40>,)

In [72]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [73]:
#예제10

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [74]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [75]:
#예제11

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [76]:
avg = make_averager()
avg(10)

10.0

In [77]:
#예제12

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [79]:
#예제13

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12319078s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00003068s] factorial(2) -> 2
[0.00007092s] factorial(3) -> 6
[0.00009816s] factorial(4) -> 24
[0.00012520s] factorial(5) -> 120
[0.00042230s] factorial(6) -> 720
6! = 720


In [80]:
#예제14

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [81]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12324166s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000143s] factorial(1) -> 1
[0.00004077s] factorial(2) -> 2
[0.00007176s] factorial(3) -> 6
[0.00009871s] factorial(4) -> 24
[0.00033545s] factorial(5) -> 120
[0.00038838s] factorial(6) -> 720
6! = 720


In [82]:
factorial.__name__

'factorial'